In [1]:
import requests # type: ignore
from urllib.parse import urljoin
import pandas as pd # type: ignore
import schedule # type: ignore
import time
from datetime import datetime, timedelta
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
from dotenv import load_dotenv


In [2]:
load_dotenv(override=True)
api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")

# VENCIMENTO DE GCP (MODELAGEM E FUNÇÕES DE ENVIO)

In [3]:
auth_url = urljoin(api_url, "/sessions")
body = {
    "nome": api_username,
    "password":api_password
}

response = requests.post(auth_url, json=body)
token = response.json()["token"]
auth_token = "Bearer " + token

# Requisição para obter os dados do Pessoas
pessoas_url = urljoin(api_url, "/pessoas?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(pessoas_url, headers=headers)
pessoas = response.json()
pessoas = pd.DataFrame(pessoas)

pessoas.head(5)

id                                     ds_nome  co_tipo_gn co_equipe_gn  \
0  192                              Emerson Aredes       704.0         None   
1  196                               Kleber Giungi       562.0         None   
2  198                          Suporte Polo Trial       664.0         None   
3  199  Dra. Caroline Cândida Carvalho de Oliveira       387.0         None   
4  200                       Dra. Tânia Caltabiano       387.0         None   

   co_formacao_maior_nivel_gn cv_ingles cv_portugues  \
0                        31.0      None         None   
1                         NaN      None         None   
2                         NaN      None         None   
3                        30.0      None         None   
4                        29.0      None         None   

  dt_ultimo_certificado_gcp registro_profissional  \
0                      None                         
1                      None                         
2                      None                  None   
3                2020-09-28            202.285 SP   
4                      None                         

  declaracao_confidencialidade  ... equipe_padrao  equipe_padrao_kits  \
0                         None  ...             1                 0.0   
1                         None  ...             1                 1.0   
2                         None  ...             0                 0.0   
3                         None  ...             1                 0.0   
4                         None  ...             0                 0.0   

   equipe_padrao_farmacia  equipe_padrao_naocega  co_externo  \
0                     0.0                    0.0        None   
1                     1.0                    0.0        None   
2                     0.0                    0.0        None   
3                     0.0                    0.0        None   
4                     0.0                    0.0        None   

                                    dados_co_tipo_gn dados_co_equipe_gn  \
0          {'id': 704, 'ds_descricao': 'Suporte TI'}               None   
1  {'id': 562, 'ds_descricao': 'Equipe padrão (ad...               None   
2             {'id': 664, 'ds_descricao': 'Suporte'}               None   
3  {'id': 387, 'ds_descricao': 'Pesquisador Princ...               None   
4  {'id': 387, 'ds_descricao': 'Pesquisador Princ...               None   

                  dados_co_formacao_maior_nivel_gn  \
0           {'id': 31, 'ds_descricao': 'Mestrado'}   
1                                             None   
2                                             None   
3     {'id': 30, 'ds_descricao': 'Especialização'}   
4  {'id': 29, 'ds_descricao': 'Superior completo'}   

                             dados_centro  \
0                                    None   
1                                    None   
2                                    None   
3  {'id': 3, 'descricao': 'Leforte HMCG'}   
4  {'id': 3, 'descricao': 'Leforte HMCG'}   

                          dados_pessoa_especialidade  
0                                                 []  
1                                                 []  
2                                                 []  
3  [{'id': 3, 'co_pessoa': 199, 'co_pessoa_especi...  
4  [{'id': 4, 'co_pessoa': 200, 'co_pessoa_especi...  

[5 rows x 22 columns]

In [4]:
gcp = pessoas[['dados_centro', 'dt_ultimo_certificado_gcp', 'ds_nome','dados_co_tipo_gn']].copy()
gcp.loc[:, 'dados_centro_id'] = gcp['dados_centro'].apply(lambda x: x['id'] if x is not None else None)
gcp.loc[:, 'dados_centro_descricao'] = gcp['dados_centro'].apply(lambda x: x['descricao'] if x is not None else None)
gcp.loc[:, 'tipo_gn'] = gcp['dados_co_tipo_gn'].apply(lambda x: x['ds_descricao'] if x is not None else None)

gcp_modelado = gcp.drop(['dados_centro', 'dados_centro_id', 'dados_co_tipo_gn'], axis=1)
gcp_modelado.head(5)

dt_ultimo_certificado_gcp                                     ds_nome  \
0                      None                              Emerson Aredes   
1                      None                               Kleber Giungi   
2                      None                          Suporte Polo Trial   
3                2020-09-28  Dra. Caroline Cândida Carvalho de Oliveira   
4                      None                       Dra. Tânia Caltabiano   

  dados_centro_descricao                                            tipo_gn  
0                   None                                         Suporte TI  
1                   None  Equipe padrão (adicionada automaticamente para...  
2                   None                                            Suporte  
3           Leforte HMCG                              Pesquisador Principal  
4           Leforte HMCG                              Pesquisador Principal

In [5]:
def verificar_vencimento_contratos(df_contratos, nome_centro, dias_para_vencimento=30):

    df_contratos['Assinatura do GCP'] = pd.to_datetime(df_contratos['Assinatura do GCP'], errors='coerce')

    hoje = datetime.today()
    limite_vencimento = hoje + timedelta(days=dias_para_vencimento)

    df_contratos['data_vencimento'] = df_contratos['Assinatura do GCP'] + timedelta(days=2 * 365)

    filtro_vencendo = (df_contratos['data_vencimento'] > hoje) & (df_contratos['data_vencimento'] <= limite_vencimento)

    filtro_sem_data = df_contratos['Assinatura do GCP'].isna()

    contratos_relevantes = df_contratos[filtro_vencendo | filtro_sem_data]

    if contratos_relevantes.empty:
        print(f"🔹 Não há contratos GCP sem data registrada no sistema ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}.")
    else:
        print(f"⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}:")
        print(contratos_relevantes[['Centro', 'Nome', 'Assinatura do GCP', 'data_vencimento']])

    return contratos_relevantes

In [6]:
hmcg = ['Leforte HMCG','Leforte Morumbi','Hospital Municipal de Barueri', 'Leforte Liberdade', 'Clinica CardialMed']
filtro1 = gcp_modelado['dados_centro_descricao'].isin(hmcg)
centro_hmcg = gcp_modelado[filtro1]
venc_gcp_hmcg = centro_hmcg[centro_hmcg['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_hmcg.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_hmcg = venc_gcp_hmcg[nova_ordem] 
venc_gcp_hmcg

/tmp/ipykernel_2153/559468711.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_hmcg.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


Centro                                        Nome  \
3     Leforte HMCG  Dra. Caroline Cândida Carvalho de Oliveira   
4     Leforte HMCG                       Dra. Tânia Caltabiano   
5     Leforte HMCG                          Dra. Tânia Navarro   
6     Leforte HMCG                       Dra. Tassia Mancillha   
14    Leforte HMCG                               Isabel Santos   
...            ...                                         ...   
3738  Leforte HMCG                    Dr. Marco Aurélio Finger   
3739  Leforte HMCG                        Dr. Jamil Calil Neto   
3740  Leforte HMCG                           Dr. Celso Higuthi   
3741  Leforte HMCG           Dr. Isaías Mendes da Silva Junior   
3742  Leforte HMCG                   Dr. Delcio Uezato Junior    

                     Função Assinatura do GCP  
3     Pesquisador Principal        2020-09-28  
4     Pesquisador Principal              None  
5     Pesquisador Principal              None  
6           Subinvestigador              None  
14    Coordenador de Estudo              None  
...                     ...               ...  
3738        Subinvestigador              None  
3739        Subinvestigador              None  
3740        Subinvestigador              None  
3741        Subinvestigador              None  
3742        Subinvestigador              None  

[215 rows x 4 columns]

In [7]:
rocio = [' Maternidade e Cirurgia Nossa Senhora do Rocio SA']
filtro2 = gcp_modelado['dados_centro_descricao'].isin(rocio)
centro_rocio = gcp_modelado[filtro2]
venc_gcp_rocio = centro_rocio [centro_rocio['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_rocio.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_rocio = venc_gcp_rocio[nova_ordem] 
venc_gcp_rocio

Empty DataFrame
Columns: [Centro, Nome, Função, Assinatura do GCP]
Index: []

In [8]:
iir_coord = ['Hospital das Clínicas de Itajubá', 'Saint-Beaute Clinique', 'Hospital Salvalus','Consultório Lopes e Sartorelli', ' Clínica Berger', 
            'QualiVida Higienópolis','Endolap Saúde' ]
filtro3 = gcp_modelado['dados_centro_descricao'].isin(iir_coord)
centro_iir_coord = gcp_modelado[filtro3]
venc_gcp_iir_coord = centro_iir_coord [centro_iir_coord['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_iir_coord.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_iir_coord = venc_gcp_iir_coord[nova_ordem] 
venc_gcp_iir_coord

/tmp/ipykernel_2153/2634511288.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_iir_coord.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


Centro  \
2024               Hospital Salvalus   
2267          QualiVida Higienópolis   
2323                   Endolap Saúde   
2324               Hospital Salvalus   
2393          QualiVida Higienópolis   
2424               Hospital Salvalus   
2598          QualiVida Higienópolis   
2607               Hospital Salvalus   
2608               Hospital Salvalus   
2655          QualiVida Higienópolis   
2663          QualiVida Higienópolis   
2696               Hospital Salvalus   
2751          QualiVida Higienópolis   
2833  Consultório Lopes e Sartorelli   
2843          QualiVida Higienópolis   
3022               Hospital Salvalus   
3050               Hospital Salvalus   
3081          QualiVida Higienópolis   
3138          QualiVida Higienópolis   
3166          QualiVida Higienópolis   
3286          QualiVida Higienópolis   
3386          QualiVida Higienópolis   
3394          QualiVida Higienópolis   
3410          QualiVida Higienópolis   

                                            Nome                 Função  \
2024                Eduardo Augusto Rabelo Socca             Biologista   
2267              Dra. Renata Tortato Meneguetti  Pesquisador Principal   
2323                   Dra. Eloá Spritze Guollo   Pesquisador Principal   
2324                          Dra. Luciana Crema  Pesquisador Principal   
2393                 Dra. Eliana Araujo da Silva  Pesquisador Principal   
2424                       Raul Alberto Valiente  Pesquisador Principal   
2598             Dra. Débora Cordeiro do Rosário  Pesquisador Principal   
2607                              Carina Carraro  Coordenador de Estudo   
2608                            Tainara Gramalio  Coordenador de Estudo   
2655          Dra. Renée Mignolo Tanaka Ferreira  Pesquisador Principal   
2663               Dr. Murillo de Araujo Martins  Pesquisador Principal   
2696                             Monanelly Silva  Coordenador de Estudo   
2751                     Dra. Liege Mentz Rosano  Pesquisador Principal   
2833                         Nathalia Westphalen          Equipe Médica   
2843                          Dra. Joyce Teodoro  Pesquisador Principal   
3022                     Dr. Marcel Menon Miyake  Pesquisador Principal   
3050                 Dra. Camilla Lopes Siqueira  Pesquisador Principal   
3081             Dr. Rafael Cruz Santana Tavares  Pesquisador Principal   
3138                      Dra. Adriana Bertolami  Pesquisador Principal   
3166                         Dra. Bruna Bighetti  Pesquisador Principal   
3286  Dra. Cinthia Leite Frizzera Borges Bognar   Pesquisador Principal   
3386         Dr. Marcus Paulo Gonçalves de Souza  Pesquisador Principal   
3394        Dr. Carlos Henrique Teixeira Andrade  Pesquisador Principal   
3410            Dr. Marcus Guilherme de Oliveira  Pesquisador Principal   

     Assinatura do GCP  
2024              None  
2267              None  
2323              None  
2324              None  
2393              None  
2424              None  
2598              None  
2607              None  
2608              None  
2655              None  
2663              None  
2696              None  
2751              None  
2833              None  
2843              None  
3022              None  
3050              None  
3081              None  
3138              None  
3166              None  
3286              None  
3386              None  
3394              None  
3410              None

In [9]:
envio_viviane = ['Hospital Pilar','Santa casa de São Paulo','Hospital São Francisco de Ribeirã Preto',
                'Hospital Antônio Prudente','Clínica CardialMed','CLINAR - Clínica de Aparelhos RespiratÃ³rios',
                'Santa Casa de Fortaleza','Hospital Vera Cruz','Hapvida','Unimed Brusque',
                'Hospital São José das Doenças Infecciosas','Otorhinus Clínica Médica','Hospital São Francisco de Araraquara',
                'Maternidade Octaviano Neves','Clínica Infectologie','Hospital Teresa de Lisieux',
                'Hospital RioMar de Belém','Hospital e Maternidade Eugênia Pinheiro',
                'Hospital São Francisco Saúde','Hospital do Coração de Campinas','Aliança Cavernoma Brasil',
                'Faculdade de Medicina de Ribeirão Preto - USP ','Centro Clínico Zona Sul ']
filtro4 = gcp_modelado['dados_centro_descricao'].isin(envio_viviane)
centro_envio_viviane = gcp_modelado[filtro4]
venc_gcp_envio_viviane = centro_envio_viviane[centro_envio_viviane['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_envio_viviane.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_envio_viviane = venc_gcp_envio_viviane[nova_ordem] 

venc_gcp_envio_viviane

/tmp/ipykernel_2153/610327026.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_envio_viviane.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


Centro                                       Nome  \
50    Santa casa de São Paulo                Dr. Giuliano Giova Volpiani   
51    Santa casa de São Paulo                        Dr. Ronaldo Rabello   
52    Santa casa de São Paulo                  Dr. Rubens José Gagliardi   
53    Santa casa de São Paulo                Dra. Walkiria Hueb Bernardi   
54         Clínica CardialMed               Dr. Heron Rhydan Saad Rached   
...                       ...                                        ...   
3590           Unimed Brusque  Dra. Angelica Cristina dos Santos Nóbrega   
3659           Unimed Brusque                         Dr. Laércio Cadore   
3673           Unimed Brusque         Dr. Luiz Fernando Silveira Parrela   
3683           Unimed Brusque              Dra. Anita Silva Brunel Alves   
3693           Unimed Brusque                  Dr. Marcus Vitor Oliveira   

                     Função Assinatura do GCP  
50    Pesquisador Principal              None  
51    Pesquisador Principal              None  
52    Pesquisador Principal              None  
53    Pesquisador Principal        2020-05-13  
54    Pesquisador Principal              None  
...                     ...               ...  
3590  Pesquisador Principal              None  
3659  Pesquisador Principal              None  
3673  Pesquisador Principal              None  
3683  Pesquisador Principal              None  
3693  Pesquisador Principal              None  

[66 rows x 4 columns]

In [10]:
stacasa_santos = ['Santa Casa de Santos' ]
filtro5 = gcp_modelado['dados_centro_descricao'].isin(stacasa_santos)
centro_stacasa_santos = gcp_modelado[filtro5]
venc_gcp_stacasa_santos = centro_stacasa_santos [centro_stacasa_santos['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_stacasa_santos.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_stacasa_santos = venc_gcp_stacasa_santos[nova_ordem]
venc_gcp_stacasa_santos

/tmp/ipykernel_2153/3690325233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_stacasa_santos.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


Centro                                      Nome  \
60    Santa Casa de Santos                 Dr. Franklein Vieira Maia   
70    Santa Casa de Santos                        Dr. Marcelo Pilnik   
80    Santa Casa de Santos                      Dr. André Sementilli   
1655  Santa Casa de Santos              Elaine Maria Borges Mancilha   
1660  Santa Casa de Santos    Dr. José Eduardo Fernandes Vasconcelos   
...                    ...                                       ...   
3545  Santa Casa de Santos                 Dr. Bruno Oliveira Santos   
3586  Santa Casa de Santos                 Dr. Leonardo Correa Silva   
3672  Santa Casa de Santos                     Dr. Igor Marijuschkin   
3709  Santa Casa de Santos             Dr. Rodrigo de Rosso e Grimas   
3735  Santa Casa de Santos  Dr. Alfredo Fernando Vecchiatti Pommella   

                     Função Assinatura do GCP  
60    Pesquisador Principal        2025-06-26  
70    Pesquisador Principal        2021-09-27  
80    Pesquisador Principal        2025-06-26  
1655  Pesquisador Principal        2025-06-26  
1660  Pesquisador Principal        2025-05-10  
...                     ...               ...  
3545        Subinvestigador              None  
3586        Subinvestigador              None  
3672  Pesquisador Principal              None  
3709        Subinvestigador              None  
3735        Subinvestigador              None  

[124 rows x 4 columns]

In [11]:
capibaribe = ['Hospital do Capibaribe']
filtro6 = gcp_modelado['dados_centro_descricao'].isin(capibaribe)
centro_capibaribe = gcp_modelado[filtro6]
venc_gcp_capibaribe = centro_capibaribe [centro_capibaribe['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_capibaribe.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_capibaribe = venc_gcp_capibaribe[nova_ordem]
venc_gcp_capibaribe

/tmp/ipykernel_2153/635713975.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_capibaribe.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


Centro                                          Nome  \
1883  Hospital do Capibaribe      Dra. Mariane Teodoro Fernandes Rodrigues   
2369  Hospital do Capibaribe           Mariane Teodoro Fernandes Rodrigues   
2481  Hospital do Capibaribe                   Leandro Apolinário da Silva   
2589  Hospital do Capibaribe                Thiago Melo Rodrigues de Mariz   
2590  Hospital do Capibaribe                 Ana Dulce Lima de Luna Freire   
2591  Hospital do Capibaribe                    Icaro Matheus Felix Santos   
2592  Hospital do Capibaribe               Carolina Natércia da Silva Lira   
2593  Hospital do Capibaribe                     Elton Menezes Gomes Silva   
2776  Hospital do Capibaribe  Dr. Arlon Breno Figueiredo Nunes da Silveira   
3231  Hospital do Capibaribe                       Ana Beatriz Evangelista   
3232  Hospital do Capibaribe                           Rúbia Dias Carvalho   
3236  Hospital do Capibaribe                  Dr. Jayme José Gouveia Filho   

                                    Função Assinatura do GCP  
1883                 Pesquisador Principal        2024-06-14  
2369                 Pesquisador Principal              None  
2481                       Subinvestigador              None  
2589                       Subinvestigador              None  
2590                       Subinvestigador              None  
2591                       Subinvestigador              None  
2592                       Subinvestigador              None  
2593                       Subinvestigador              None  
2776                         Equipe Médica        2023-10-30  
3231  Estagiário/Assistente de Coordenação        2024-08-05  
3232                          Farmacêutico              None  
3236                         Equipe Médica        2024-09-23

In [12]:
contratos_hmcg = verificar_vencimento_contratos(venc_gcp_hmcg, "HMCG", dias_para_vencimento=30)
contratos_hmcg

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para HMCG:
            Centro                               Nome Assinatura do GCP  \
4     Leforte HMCG              Dra. Tânia Caltabiano               NaT   
5     Leforte HMCG                 Dra. Tânia Navarro               NaT   
6     Leforte HMCG              Dra. Tassia Mancillha               NaT   
14    Leforte HMCG                      Isabel Santos               NaT   
15    Leforte HMCG                      Giovana Rosas               NaT   
...            ...                                ...               ...   
3738  Leforte HMCG           Dr. Marco Aurélio Finger               NaT   
3739  Leforte HMCG               Dr. Jamil Calil Neto               NaT   
3740  Leforte HMCG                  Dr. Celso Higuthi               NaT   
3741  Leforte HMCG  Dr. Isaías Mendes da Silva Junior               NaT   
3742  Leforte HMCG          Dr. Delcio Uezato Junior                NaT   

 

Centro                               Nome                 Função  \
4     Leforte HMCG              Dra. Tânia Caltabiano  Pesquisador Principal   
5     Leforte HMCG                 Dra. Tânia Navarro  Pesquisador Principal   
6     Leforte HMCG              Dra. Tassia Mancillha        Subinvestigador   
14    Leforte HMCG                      Isabel Santos  Coordenador de Estudo   
15    Leforte HMCG                      Giovana Rosas                   None   
...            ...                                ...                    ...   
3738  Leforte HMCG           Dr. Marco Aurélio Finger        Subinvestigador   
3739  Leforte HMCG               Dr. Jamil Calil Neto        Subinvestigador   
3740  Leforte HMCG                  Dr. Celso Higuthi        Subinvestigador   
3741  Leforte HMCG  Dr. Isaías Mendes da Silva Junior        Subinvestigador   
3742  Leforte HMCG          Dr. Delcio Uezato Junior         Subinvestigador   

     Assinatura do GCP data_vencimento  
4                  NaT             NaT  
5                  NaT             NaT  
6                  NaT             NaT  
14                 NaT             NaT  
15                 NaT             NaT  
...                ...             ...  
3738               NaT             NaT  
3739               NaT             NaT  
3740               NaT             NaT  
3741               NaT             NaT  
3742               NaT             NaT  

[204 rows x 5 columns]

In [13]:
contratos_rocio = verificar_vencimento_contratos(venc_gcp_rocio, "Rocío", dias_para_vencimento=30)

🔹 Não há contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Rocío.


In [14]:
contratos_iir_coord = verificar_vencimento_contratos(venc_gcp_iir_coord, "IRR COORD", dias_para_vencimento=30)
contratos_iir_coord

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para IRR COORD:
                              Centro  \
2024               Hospital Salvalus   
2267          QualiVida Higienópolis   
2323                   Endolap Saúde   
2324               Hospital Salvalus   
2393          QualiVida Higienópolis   
2424               Hospital Salvalus   
2598          QualiVida Higienópolis   
2607               Hospital Salvalus   
2608               Hospital Salvalus   
2655          QualiVida Higienópolis   
2663          QualiVida Higienópolis   
2696               Hospital Salvalus   
2751          QualiVida Higienópolis   
2833  Consultório Lopes e Sartorelli   
2843          QualiVida Higienópolis   
3022               Hospital Salvalus   
3050               Hospital Salvalus   
3081          QualiVida Higienópolis   
3138          QualiVida Higienópolis   
3166          QualiVida Higienópolis   
3286          QualiVida Higienópolis   
3386          Qua

Centro  \
2024               Hospital Salvalus   
2267          QualiVida Higienópolis   
2323                   Endolap Saúde   
2324               Hospital Salvalus   
2393          QualiVida Higienópolis   
2424               Hospital Salvalus   
2598          QualiVida Higienópolis   
2607               Hospital Salvalus   
2608               Hospital Salvalus   
2655          QualiVida Higienópolis   
2663          QualiVida Higienópolis   
2696               Hospital Salvalus   
2751          QualiVida Higienópolis   
2833  Consultório Lopes e Sartorelli   
2843          QualiVida Higienópolis   
3022               Hospital Salvalus   
3050               Hospital Salvalus   
3081          QualiVida Higienópolis   
3138          QualiVida Higienópolis   
3166          QualiVida Higienópolis   
3286          QualiVida Higienópolis   
3386          QualiVida Higienópolis   
3394          QualiVida Higienópolis   
3410          QualiVida Higienópolis   

                                            Nome                 Função  \
2024                Eduardo Augusto Rabelo Socca             Biologista   
2267              Dra. Renata Tortato Meneguetti  Pesquisador Principal   
2323                   Dra. Eloá Spritze Guollo   Pesquisador Principal   
2324                          Dra. Luciana Crema  Pesquisador Principal   
2393                 Dra. Eliana Araujo da Silva  Pesquisador Principal   
2424                       Raul Alberto Valiente  Pesquisador Principal   
2598             Dra. Débora Cordeiro do Rosário  Pesquisador Principal   
2607                              Carina Carraro  Coordenador de Estudo   
2608                            Tainara Gramalio  Coordenador de Estudo   
2655          Dra. Renée Mignolo Tanaka Ferreira  Pesquisador Principal   
2663               Dr. Murillo de Araujo Martins  Pesquisador Principal   
2696                             Monanelly Silva  Coordenador de Estudo   
2751                     Dra. Liege Mentz Rosano  Pesquisador Principal   
2833                         Nathalia Westphalen          Equipe Médica   
2843                          Dra. Joyce Teodoro  Pesquisador Principal   
3022                     Dr. Marcel Menon Miyake  Pesquisador Principal   
3050                 Dra. Camilla Lopes Siqueira  Pesquisador Principal   
3081             Dr. Rafael Cruz Santana Tavares  Pesquisador Principal   
3138                      Dra. Adriana Bertolami  Pesquisador Principal   
3166                         Dra. Bruna Bighetti  Pesquisador Principal   
3286  Dra. Cinthia Leite Frizzera Borges Bognar   Pesquisador Principal   
3386         Dr. Marcus Paulo Gonçalves de Souza  Pesquisador Principal   
3394        Dr. Carlos Henrique Teixeira Andrade  Pesquisador Principal   
3410            Dr. Marcus Guilherme de Oliveira  Pesquisador Principal   

     Assinatura do GCP data_vencimento  
2024               NaT             NaT  
2267               NaT             NaT  
2323               NaT             NaT  
2324               NaT             NaT  
2393               NaT             NaT  
2424               NaT             NaT  
2598               NaT             NaT  
2607               NaT             NaT  
2608               NaT             NaT  
2655               NaT             NaT  
2663               NaT             NaT  
2696               NaT             NaT  
2751               NaT             NaT  
2833               NaT             NaT  
2843               NaT             NaT  
3022               NaT             NaT  
3050               NaT             NaT  
3081               NaT             NaT  
3138               NaT             NaT  
3166               NaT             NaT  
3286               NaT             NaT  
3386               NaT             NaT  
3394               NaT             NaT  
3410               NaT             NaT

In [15]:
contratos_vivi = verificar_vencimento_contratos(venc_gcp_envio_viviane, "Envio Viviane", dias_para_vencimento=30)
contratos_vivi

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Envio Viviane:
                         Centro                                       Nome  \
50      Santa casa de São Paulo                Dr. Giuliano Giova Volpiani   
51      Santa casa de São Paulo                        Dr. Ronaldo Rabello   
52      Santa casa de São Paulo                  Dr. Rubens José Gagliardi   
54           Clínica CardialMed               Dr. Heron Rhydan Saad Rached   
73    Hospital Antônio Prudente                       Dr. Bruno Cavalcante   
...                         ...                                        ...   
3590             Unimed Brusque  Dra. Angelica Cristina dos Santos Nóbrega   
3659             Unimed Brusque                         Dr. Laércio Cadore   
3673             Unimed Brusque         Dr. Luiz Fernando Silveira Parrela   
3683             Unimed Brusque              Dra. Anita Silva Brunel Alves   
3693             Unimed Brusque    

Centro                                       Nome  \
50      Santa casa de São Paulo                Dr. Giuliano Giova Volpiani   
51      Santa casa de São Paulo                        Dr. Ronaldo Rabello   
52      Santa casa de São Paulo                  Dr. Rubens José Gagliardi   
54           Clínica CardialMed               Dr. Heron Rhydan Saad Rached   
73    Hospital Antônio Prudente                       Dr. Bruno Cavalcante   
...                         ...                                        ...   
3590             Unimed Brusque  Dra. Angelica Cristina dos Santos Nóbrega   
3659             Unimed Brusque                         Dr. Laércio Cadore   
3673             Unimed Brusque         Dr. Luiz Fernando Silveira Parrela   
3683             Unimed Brusque              Dra. Anita Silva Brunel Alves   
3693             Unimed Brusque                  Dr. Marcus Vitor Oliveira   

                     Função Assinatura do GCP data_vencimento  
50    Pesquisador Principal               NaT             NaT  
51    Pesquisador Principal               NaT             NaT  
52    Pesquisador Principal               NaT             NaT  
54    Pesquisador Principal               NaT             NaT  
73    Pesquisador Principal               NaT             NaT  
...                     ...               ...             ...  
3590  Pesquisador Principal               NaT             NaT  
3659  Pesquisador Principal               NaT             NaT  
3673  Pesquisador Principal               NaT             NaT  
3683  Pesquisador Principal               NaT             NaT  
3693  Pesquisador Principal               NaT             NaT  

[63 rows x 5 columns]

In [16]:
contratos_stacsantos = verificar_vencimento_contratos(venc_gcp_stacasa_santos, "Santa Casa de Santos", dias_para_vencimento=30)
contratos_stacsantos

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Santa Casa de Santos:
                    Centro                                      Nome  \
1663  Santa Casa de Santos                           Elaine Mancilha   
1664  Santa Casa de Santos                      Dra. Elaine Mancilha   
2183  Santa Casa de Santos                    Dr. Fabio Crescentini    
2193  Santa Casa de Santos                   Dr. Roberto Higa Junior   
2221  Santa Casa de Santos                    Eliana Araújo da Silva   
...                    ...                                       ...   
3545  Santa Casa de Santos                 Dr. Bruno Oliveira Santos   
3586  Santa Casa de Santos                 Dr. Leonardo Correa Silva   
3672  Santa Casa de Santos                     Dr. Igor Marijuschkin   
3709  Santa Casa de Santos             Dr. Rodrigo de Rosso e Grimas   
3735  Santa Casa de Santos  Dr. Alfredo Fernando Vecchiatti Pommella   

     Assinatura do GC

Centro                                      Nome  \
1663  Santa Casa de Santos                           Elaine Mancilha   
1664  Santa Casa de Santos                      Dra. Elaine Mancilha   
2183  Santa Casa de Santos                    Dr. Fabio Crescentini    
2193  Santa Casa de Santos                   Dr. Roberto Higa Junior   
2221  Santa Casa de Santos                    Eliana Araújo da Silva   
...                    ...                                       ...   
3545  Santa Casa de Santos                 Dr. Bruno Oliveira Santos   
3586  Santa Casa de Santos                 Dr. Leonardo Correa Silva   
3672  Santa Casa de Santos                     Dr. Igor Marijuschkin   
3709  Santa Casa de Santos             Dr. Rodrigo de Rosso e Grimas   
3735  Santa Casa de Santos  Dr. Alfredo Fernando Vecchiatti Pommella   

                     Função Assinatura do GCP data_vencimento  
1663  Pesquisador Principal               NaT             NaT  
1664  Pesquisador Principal               NaT             NaT  
2183  Pesquisador Principal               NaT             NaT  
2193  Pesquisador Principal               NaT             NaT  
2221  Pesquisador Principal               NaT             NaT  
...                     ...               ...             ...  
3545        Subinvestigador               NaT             NaT  
3586        Subinvestigador               NaT             NaT  
3672  Pesquisador Principal               NaT             NaT  
3709        Subinvestigador               NaT             NaT  
3735        Subinvestigador               NaT             NaT  

[76 rows x 5 columns]

In [17]:
contratos_capiberibe = verificar_vencimento_contratos(venc_gcp_capibaribe, "Capiberibe", dias_para_vencimento=30)
contratos_capiberibe

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Capiberibe:
                      Centro                                 Nome  \
2369  Hospital do Capibaribe  Mariane Teodoro Fernandes Rodrigues   
2481  Hospital do Capibaribe          Leandro Apolinário da Silva   
2589  Hospital do Capibaribe       Thiago Melo Rodrigues de Mariz   
2590  Hospital do Capibaribe        Ana Dulce Lima de Luna Freire   
2591  Hospital do Capibaribe           Icaro Matheus Felix Santos   
2592  Hospital do Capibaribe      Carolina Natércia da Silva Lira   
2593  Hospital do Capibaribe            Elton Menezes Gomes Silva   
3232  Hospital do Capibaribe                  Rúbia Dias Carvalho   

     Assinatura do GCP data_vencimento  
2369               NaT             NaT  
2481               NaT             NaT  
2589               NaT             NaT  
2590               NaT             NaT  
2591               NaT             NaT  
2592               NaT      

Centro                                 Nome  \
2369  Hospital do Capibaribe  Mariane Teodoro Fernandes Rodrigues   
2481  Hospital do Capibaribe          Leandro Apolinário da Silva   
2589  Hospital do Capibaribe       Thiago Melo Rodrigues de Mariz   
2590  Hospital do Capibaribe        Ana Dulce Lima de Luna Freire   
2591  Hospital do Capibaribe           Icaro Matheus Felix Santos   
2592  Hospital do Capibaribe      Carolina Natércia da Silva Lira   
2593  Hospital do Capibaribe            Elton Menezes Gomes Silva   
3232  Hospital do Capibaribe                  Rúbia Dias Carvalho   

                     Função Assinatura do GCP data_vencimento  
2369  Pesquisador Principal               NaT             NaT  
2481        Subinvestigador               NaT             NaT  
2589        Subinvestigador               NaT             NaT  
2590        Subinvestigador               NaT             NaT  
2591        Subinvestigador               NaT             NaT  
2592        Subinvestigador               NaT             NaT  
2593        Subinvestigador               NaT             NaT  
3232           Farmacêutico               NaT             NaT

In [18]:
css_hover = "<style>table {border-collapse: collapse; width: 100%;} th, td {border: 1px solid black; padding: 8px; text-align: left;} th {background-color: #007bff;color: white;} tr:hover {background-color: #fff59d; box-shadow: inset 0 0 10px rgba(255, 235, 59, 0.5);}</style>"

In [19]:
smtp_server = os.getenv("EMAIL_SERVER")
email_port = int(os.getenv("EMAIL_PORT"))
email_usuario = os.getenv("EMAIL_USERNAME")
email_senha = os.getenv("EMAIL_PASSWORD")
enviar_para = os.getenv('ENVIAR_PARA')
destinatario_hmcg = os.getenv('DESTINATARIO_HMCG')
destinatario_rocio =  os.getenv('DESTINATARIO_ROCIO')
destinatario_scs = os.getenv('DESTINATARIO_SCS')
destinatario_iir =  os.getenv('DESTINATARIO_IIR')
destinatario_vivi =  os.getenv('DESTINATARIO_VIVIANE')
destinatario_capiberibe =  os.getenv('DESTINATARIO_CAPIBERIBE')

In [20]:
def enviar_email(destinatário,df_tratado):
    try:
        if df_tratado is None or df_tratado.empty:
            print("Não há contratos vencidos para enviar por e-mail.")
            return

        tabela_html = df_tratado[['Centro', 'Nome', 'Assinatura do GCP', 'data_vencimento']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Bcc'] = ', '.join(destinatário)
        msg['Subject'] = "GCP com vencimento nos próximos 30 dias ou sem data informada - "

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <h2>Vencimento de GCP</h2>
                <p>Olá,</p> 
                <br>
                <p>Segue abaixo GCP que vencerão no próximo mês ou sem data informada no sistema:</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridadas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port
    ) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            server.send_message(msg)

        print("E-mail de GCP que já venceram enviado com sucesso!")
    
    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

In [21]:
enviar_email([destinatario_hmcg], contratos_hmcg)
enviar_email([destinatario_rocio], contratos_rocio)
enviar_email([destinatario_iir], contratos_iir_coord)
enviar_email([destinatario_scs], contratos_stacsantos)
enviar_email([destinatario_vivi], contratos_vivi)
enviar_email([destinatario_capiberibe], contratos_capiberibe)

E-mail de GCP que já venceram enviado com sucesso!
Não há contratos vencidos para enviar por e-mail.


Erro ao enviar o e-mail: 'utf8' is an invalid keyword argument for Compat32


E-mail de GCP que já venceram enviado com sucesso!


E-mail de GCP que já venceram enviado com sucesso!


E-mail de GCP que já venceram enviado com sucesso!


# VISITAS DE SEGUIMENTO (MODELAGEM E FUNÇÕES DE ENVIO)

In [22]:
protocolo_url = urljoin(api_url, "/protocolo?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(protocolo_url, headers=headers)
protocolo = response.json()
protocolo = pd.DataFrame(protocolo)
protocolo.head(2)

id                                   titulo_protocolo cor_agenda  \
0   4  ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...    #330eb9   
1   5  ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...    #330eb9   

  numero_protocolo apelido_protocolo  coordenador     pi  pesquisador_backup  \
0    BTK-COV-202BR               BTK        261.0  306.0               210.0   
1    BTK-COV-202BR               BTK        310.0  199.0               227.0   

   co_pessoa_regulatorio             data_cadastro  ...  \
0                  214.0  2021-04-05T18:37:50.000Z  ...   
1                  214.0  2021-07-07T12:28:33.000Z  ...   

                 dados_aprovacao_cep  \
0  {'id': 60, 'ds_descricao': 'Sim'}   
1  {'id': 60, 'ds_descricao': 'Sim'}   

                               dados_co_centro  \
0       {'id': 3, 'descricao': 'Leforte HMCG'}   
1  {'id': 2, 'descricao': 'Leforte Liberdade'}   

                                            PessoaPI  \
0  {'id': 306, 'ds_nome': 'Dr. Carlos Augusto Qua...   
1  {'id': 199, 'ds_nome': 'Dra. Caroline Cândida ...   

                                   PessoaCoordenador  \
0            {'id': 261, 'ds_nome': 'Giovana Rosas'}   
1  {'id': 310, 'ds_nome': 'Isabel Cristina dos Sa...   

                                   PessoaPesquisador  \
0                                               None   
1  {'id': 227, 'ds_nome': 'Viviane Santana da Sil...   

                                   PessoaRegulatorio  \
0  {'id': 214, 'ds_nome': 'Priscila do Prado Costa'}   
1  {'id': 214, 'ds_nome': 'Priscila do Prado Costa'}   

                             fase_pesquisa  \
0  {'id': 123, 'ds_descricao': 'Fase III'}   
1   {'id': 122, 'ds_descricao': 'Fase II'}   

                                     status  \
0  {'id': 296, 'ds_descricao': 'Concluído'}   
1  {'id': 296, 'ds_descricao': 'Concluído'}   

                               tipo_iniciativa  \
0  {'id': 506, 'ds_descricao': 'Patrocinador'}   
1  {'id': 506, 'ds_descricao': 'Patrocinador'}   

                                   nome_patrocinador  
0  {'id': 723, 'ds_descricao': 'Sorrento Therapeu...  
1  {'id': 723, 'ds_descricao': 'Sorrento Therapeu...  

[2 rows x 124 columns]

In [23]:
centros = protocolo[['id','apelido_protocolo', 'numero_protocolo','co_externo','apelido_centro']].copy()
centros

id   apelido_protocolo numero_protocolo     co_externo  \
0        4                 BTK    BTK-COV-202BR  BTK-COV-202BR   
1        5                 BTK    BTK-COV-202BR  BTK-COV-202BR   
2        8         RIGEL-FOCUS     C-935788-061   C-935788-061   
3        9         BIOTEST-998     ESsCOVID-998           None   
4       10              GARDEN       ACT-CS-006     ACT-CS-006   
...    ...                 ...              ...            ...   
2336  2547   Chron e obesidade                            None   
2337  2548   Chron e obesidade                            None   
2338  2549  Transplante de Rim                            None   
2339  2550      R9933-PAD-2394                            None   
2340  2551         Oxandrolona                            None   

                                         apelido_centro  
0                                    BTK - Leforte HMCG  
1                               BTK - Leforte Liberdade  
2                       RIGEL-FOCUS - Leforte Liberdade  
3                            BIOTEST-998 - Leforte HMCG  
4                                 GARDEN - Leforte HMCG  
...                                                 ...  
2336             Chron e obesidade - Clínica CardialMed  
2337           Chron e obesidade - Santa Casa de Santos  
2338  Transplante de Rim - Maternidade e Cirurgia No...  
2339                      R9933-PAD-2394 - Leforte HMCG  
2340                  Oxandrolona - Centro Vila Olimpia  

[2341 rows x 5 columns]

In [24]:
participantes_visita_url = urljoin(api_url, "/participante_visita?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(participantes_visita_url, headers=headers)
participantes_visita = response.json()
participantes_visita = pd.DataFrame(participantes_visita)

participantes_visita.head(2)

id  co_participante  co_visita  status_participante_visita  \
0  639               54        1.0                          20   
1  709               54        2.0                          20   

          nome_tarefa             data_estimada data_estimada_fim  \
0            Triagem   2021-01-05T00:00:00.000Z              None   
1  Rand + Dose 1 - D1  2021-01-06T00:00:00.000Z              None   

             data_realizada observacoes valor_previsto  ...  co_externo  \
0  2021-01-05T00:00:00.000Z        None        7940.48  ...        None   
1  2021-01-06T00:00:00.000Z        None        6484.08  ...        None   

   data_preenchimento_visita  data_preenchimento_crf agendada_usuario  \
0                       None                    None             None   
1                       None                    None             None   

                                  dados_participante  \
0  {'id': 54, 'co_protocolo': 4, 'id_participante...   
1  {'id': 54, 'co_protocolo': 4, 'id_participante...   

                                        dados_visita  \
0  {'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...   
1  {'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...   

                              dados_status  \
0  {'id': 20, 'ds_descricao': 'Realizada'}   
1  {'id': 20, 'ds_descricao': 'Realizada'}   

                             dados_nota_fiscal dados_local dados_responsavel  
0  {'id': 784, 'codigo_nota_fiscal': 'NF 171'}        None              None  
1  {'id': 784, 'codigo_nota_fiscal': 'NF 171'}        None              None  

[2 rows x 23 columns]

In [25]:
seguimento = participantes_visita[['dados_participante', 'data_estimada', 'dados_status', 'dados_visita']].copy()

seguimento.loc[:, 'dados_participante_id'] = seguimento['dados_participante'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'id_participante'] = seguimento['dados_participante'].apply(lambda x: x['id_participante'] if x is not None else None)
seguimento.loc[:, 'dados_protocolo'] = seguimento['dados_participante'].apply(lambda x: x['dados_protocolo'] if x is not None else None)

seguimento.loc[:, 'dados_status_id'] = seguimento['dados_status'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'ds_descricao'] = seguimento['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)
seguimento.loc[:, 'ds_nome_visita'] = seguimento['dados_visita'].apply(lambda x: x['ds_nome_visita'] if x is not None else None)

seguimento.loc[:, 'apelido_protocolo'] = seguimento['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)
seguimento['data_estimada'] = pd.to_datetime(seguimento['data_estimada']).dt.date
seguimento.head(2)

dados_participante data_estimada  \
0  {'id': 54, 'co_protocolo': 4, 'id_participante...    2021-01-05   
1  {'id': 54, 'co_protocolo': 4, 'id_participante...    2021-01-06   

                              dados_status  \
0  {'id': 20, 'ds_descricao': 'Realizada'}   
1  {'id': 20, 'ds_descricao': 'Realizada'}   

                                        dados_visita  dados_participante_id  \
0  {'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...                     54   
1  {'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...                     54   

  id_participante                        dados_protocolo  dados_status_id  \
0            1001  {'id': 4, 'apelido_protocolo': 'BTK'}               20   
1            1001  {'id': 4, 'apelido_protocolo': 'BTK'}               20   

  ds_descricao      ds_nome_visita apelido_protocolo  
0    Realizada            Triagem                BTK  
1    Realizada  Rand + Dose 1 - D1               BTK

In [26]:
seguimento_tratado = seguimento.drop(['dados_participante','dados_status','id_participante','dados_status_id'], axis=1)
seguimento_tratado.loc[:, 'id'] = seguimento['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
seguimento_tratado.head(2)

data_estimada                                       dados_visita  \
0    2021-01-05  {'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...   
1    2021-01-06  {'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...   

   dados_participante_id                        dados_protocolo ds_descricao  \
0                     54  {'id': 4, 'apelido_protocolo': 'BTK'}    Realizada   
1                     54  {'id': 4, 'apelido_protocolo': 'BTK'}    Realizada   

       ds_nome_visita apelido_protocolo  id  
0            Triagem                BTK   4  
1  Rand + Dose 1 - D1               BTK   4

In [27]:
seguimentos = pd.merge(centros, seguimento_tratado, on='id', how='inner')
seguimentos

id       apelido_protocolo_x                       numero_protocolo  \
0         4                       BTK                          BTK-COV-202BR   
1         4                       BTK                          BTK-COV-202BR   
2         4                       BTK                          BTK-COV-202BR   
3         4                       BTK                          BTK-COV-202BR   
4         4                       BTK                          BTK-COV-202BR   
...     ...                       ...                                    ...   
15876  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
15877  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
15878  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
15879  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
15880  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   

          co_externo                                  apelido_centro  \
0      BTK-COV-202BR                              BTK - Leforte HMCG   
1      BTK-COV-202BR                              BTK - Leforte HMCG   
2      BTK-COV-202BR                              BTK - Leforte HMCG   
3      BTK-COV-202BR                              BTK - Leforte HMCG   
4      BTK-COV-202BR                              BTK - Leforte HMCG   
...              ...                                             ...   
15876           None  Post Market Surveillance - Centro Vila Olimpia   
15877           None  Post Market Surveillance - Centro Vila Olimpia   
15878           None  Post Market Surveillance - Centro Vila Olimpia   
15879           None  Post Market Surveillance - Centro Vila Olimpia   
15880           None  Post Market Surveillance - Centro Vila Olimpia   

      data_estimada                                       dados_visita  \
0        2021-01-05  {'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...   
1        2021-01-06  {'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...   
2        2021-01-07  {'id': 7, 'ds_nome_visita': 'D2 - Hospitalizaç...   
3        2021-01-08  {'id': 8, 'ds_nome_visita': 'D3 - Hospitalizaç...   
4        2021-01-09  {'id': 9, 'ds_nome_visita': 'D4 - Hospitalizaç...   
...             ...                                                ...   
15876    2025-09-12  {'id': 2980, 'ds_nome_visita': 'Questionário S...   
15877    2025-09-12  {'id': 2980, 'ds_nome_visita': 'Questionário S...   
15878    2025-09-12  {'id': 2980, 'ds_nome_visita': 'Questionário S...   
15879    2025-09-12  {'id': 2980, 'ds_nome_visita': 'Questionário S...   
15880    2025-09-12  {'id': 2980, 'ds_nome_visita': 'Questionário S...   

       dados_participante_id  \
0                         54   
1                         54   
2                         54   
3                         54   
4                         54   
...                      ...   
15876                   3249   
15877                   3250   
15878                   3251   
15879                   3252   
15880                   3253   

                                         dados_protocolo ds_descricao  \
0                  {'id': 4, 'apelido_protocolo': 'BTK'}    Realizada   
1                  {'id': 4, 'apelido_protocolo': 'BTK'}    Realizada   
2                  {'id': 4, 'apelido_protocolo': 'BTK'}    Realizada   
3                  {'id': 4, 'apelido_protocolo': 'BTK'}    Realizada   
4                  {'id': 4, 'apelido_protocolo': 'BTK'}    Realizada   
...                                                  ...          ...   
15876  {'id': 2057, 'apelido_protocolo': 'Post Market...    Realizada   
15877  {'id': 2057, 'apelido_protocolo': 'Post Market...    Realizada   
15878  {'id': 2057, 'apelido_protocolo': 'Post Market...    Realizada   
15879  {'id': 2057, 'apelido_protocolo': 'Post Market...    Realizada   
15880  {'id': 2057, 'apelido_protocolo': 'Post Market...    Realizada   

             ds_nome_visita     

In [28]:
seguimentos['apelido_protocolo_x'] = seguimentos['apelido_protocolo_x'].str.strip()
visitas_filtrado = seguimentos[seguimentos['ds_descricao'].str.contains('Pendente')]
nova_ordem = ["apelido_centro", "dados_participante_id", "ds_nome_visita","data_estimada","ds_descricao"]
visitas_reordenado = visitas_filtrado[nova_ordem]
visitas_reordenado.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_participante_id': 'ID Participante', 'ds_nome_visita': 'Tipo Visita',
                          'data_estimada':'Data Estimada', 'ds_descricao':'Status'}, inplace=True)
visitas_reordenado

/tmp/ipykernel_2153/292463722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visitas_reordenado.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_participante_id': 'ID Participante', 'ds_nome_visita': 'Tipo Visita',


Estudo/Centro  ID Participante  \
5338                VICTORION 2 - PREVENT - Leforte HMCG             2079   
5339                VICTORION 2 - PREVENT - Leforte HMCG             2079   
5350                VICTORION 2 - PREVENT - Leforte HMCG             2079   
5351                VICTORION 2 - PREVENT - Leforte HMCG             2079   
5352                VICTORION 2 - PREVENT - Leforte HMCG             2079   
...                                                  ...              ...   
15724  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...             3227   
15725  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...             3227   
15726  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...             3227   
15727  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...             3227   
15728  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...             3227   

              Tipo Visita Data Estimada    Status  
5338                  EOS           NaT  Pendente  
5339    Visita Unschedule           NaT  Pendente  
5350               Mês 39    2026-02-27  Pendente  
5351               Mês 45    2026-08-26  Pendente  
5352               Mês 51    2027-02-22  Pendente  
...                   ...           ...       ...  
15724                 V26    2025-09-30  Pendente  
15725                 V27    2025-10-01  Pendente  
15726                 V28    2025-10-02  Pendente  
15727     Fim do FU / D29    2025-10-03  Pendente  
15728  Encerramento / D91    2025-12-04  Pendente  

[3620 rows x 5 columns]

In [29]:
datas_visitas = visitas_reordenado.dropna(subset=["Data Estimada"])
datas_visitas = datas_visitas.sort_values(by= "Data Estimada", ascending=True)
datas_visitas = datas_visitas[~datas_visitas["Tipo Visita"].isin(["Unscheduled", "Triagem", "End of Study"])]
valores_contagem = datas_visitas['Tipo Visita'].value_counts()
datas_visitas

Estudo/Centro  ID Participante  \
5970                VICTORION 2 - PREVENT - Leforte HMCG             2178   
6577                VICTORION 2 - PREVENT - Leforte HMCG             2339   
6637                VICTORION 2 - PREVENT - Leforte HMCG             2350   
12255                         MK0616-015  - Leforte HMCG             3108   
12642  EASi-HF - 1378-0020 - Maternidade e Cirurgia N...             3254   
...                                                  ...              ...   
11315                 VICTORION-1-PREVENT - Leforte HMCG             2663   
11296                 VICTORION-1-PREVENT - Leforte HMCG             2662   
11353                 VICTORION-1-PREVENT - Leforte HMCG             2683   
11334                 VICTORION-1-PREVENT - Leforte HMCG             2682   
11372                 VICTORION-1-PREVENT - Leforte HMCG             2687   

                  Tipo Visita Data Estimada    Status  
5970                   Mês 33    2025-09-14  Pendente  
6577                   Mês 27    2025-09-15  Pendente  
6637                   Mês 27    2025-09-17  Pendente  
12255                    None    2025-09-17  Pendente  
12642  Randomização/ Visita 2    2025-09-17  Pendente  
...                       ...           ...       ...  
11315               V14 - M75    2030-06-08  Pendente  
11296               V14 - M75    2030-06-08  Pendente  
11353               V14 - M75    2030-07-01  Pendente  
11334               V14 - M75    2030-07-01  Pendente  
11372               V14 - M75    2030-07-01  Pendente  

[2554 rows x 5 columns]

In [30]:
def verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20):
    datas_visitas['Data Estimada'] = pd.to_datetime(datas_visitas['Data Estimada'])
    
    hoje = datetime.today()
    limite_visita = hoje + timedelta(days=dias_para_visita)
    
    visitas_futuras = datas_visitas[
        (datas_visitas['Data Estimada'] >= hoje) & 
        (datas_visitas['Data Estimada'] <= limite_visita)
    ]
    
    if visitas_futuras.empty:
        print(f"Não há visitas programadas para os próximos {dias_para_visita} dias.")
    else:
        print(f"Visitas programadas para os próximos {dias_para_visita} dias:")
        print(visitas_futuras[['Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada','Status']])
    
    return visitas_futuras

visitas_20_dias = verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20)
# próximas_visitas = visitas_20_dias.drop(['dados_visita'], axis=1)
# próximas_visitas.head(2)

Visitas programadas para os próximos 20 dias:
                                           Estudo/Centro  ID Participante  \
6431                VICTORION 2 - PREVENT - Leforte HMCG             2320   
10679                 VICTORION-1-PREVENT - Leforte HMCG             2465   
15713  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...             3227   
11228                 VICTORION-1-PREVENT - Leforte HMCG             2627   
6097                VICTORION 2 - PREVENT - Leforte HMCG             2190   
...                                                  ...              ...   
15344  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2976   
10885                 VICTORION-1-PREVENT - Leforte HMCG             2535   
10866                 VICTORION-1-PREVENT - Leforte HMCG             2534   
10846                 VICTORION-1-PREVENT - Leforte HMCG             2526   
10904                 VICTORION-1-PREVENT - Leforte HMCG             2542   

      Tipo Visita Data Estima

In [31]:
def enviar_emails(próximas_visitas):
    global enviar_para  # Acessa a variável global 'enviar_para'
    
    try:
        if próximas_visitas is None or próximas_visitas.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        colunas_esperadas = {'Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada', 'Status'}
        if not colunas_esperadas.issubset(próximas_visitas.columns):
            print("Erro: DataFrame não contém todas as colunas esperadas.")
            return

        if not all([smtp_server, email_usuario, email_senha, email_port]):
            print("Erro: Configurações de e-mail estão incompletas.")
            return

        # Validação de e-mails com regex
        email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        if isinstance(enviar_para, list):
            enviar_para = [email.strip() for email in enviar_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            enviar_para = []

        # Criação da tabela HTML
        tabela_html = próximas_visitas[list(colunas_esperadas)].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        # Montagem da mensagem
        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Seguimento - próximos 20 dias"
        
        # Define o campo BCC apenas se houver destinatários válidos
        if enviar_para:
            msg['Bcc'] = ', '.join(enviar_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de seguimento programadas para os próximos 20 dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        # Envio de e-mail
        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            
            # Lista final de destinatários
            destinatarios = [email_usuario] + enviar_para
            if destinatarios:
                server.sendmail(email_usuario, destinatarios, msg.as_string())
                print("E-mail de visitas pendentes enviado com sucesso!")
            else:
                print("Nenhum destinatário válido. E-mail não enviado.")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(visitas_20_dias)

E-mail de visitas pendentes enviado com sucesso!


# VISITAS DE MONITORIA (MODELAGEM E FUNÇÃO ENVIO DE EMAIL)

In [32]:
agenda_url = urljoin(api_url, "/agenda?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(agenda_url, headers=headers)
agenda = response.json()
agenda = pd.DataFrame(agenda)

agenda.head(2)

id  tipo_evento       titulo ds_co_industria ds_co_hc co_participante  \
0   1            1   Centriguga                                            
1   4            1  contrato V6                                            

   tipo  status_evento             data_estimada data_estimada_fim  ...  \
0     4             20  2022-04-28T00:00:00.000Z              None  ...   
1     6             20  2022-07-06T00:00:00.000Z              None  ...   

                data_agenda    color        title  co_visita_referencia  \
0  2023-08-23T00:00:00.000Z  #428F54   Centriguga                  None   
1  2022-07-06T00:00:00.000Z  #000000  contrato V6                  None   

  filtro_dias  co_externo                       dados_tipo  \
0      1239.0        None  {'id': 4, 'tipo': 'Calibração'}   
1      1170.0        None    {'id': 6, 'tipo': 'Contrato'}   

                              dados_status  \
0  {'id': 20, 'ds_descricao': 'Realizada'}   
1  {'id': 20, 'ds_descricao': 'Realizada'}   

                            dados_protocolo                 dados_tipo_evento  
0                                      None  {'id': 1, 'descricao': 'Evento'}  
1  {'id': 53, 'apelido_protocolo': 'Dipro'}  {'id': 1, 'descricao': 'Evento'}  

[2 rows x 38 columns]

In [33]:
monitoria = agenda[['dados_protocolo', 'dados_tipo', 'dados_status', 'data_estimada_filter']].copy()

monitoria.loc[:, 'dados_protocolo_id'] = monitoria['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'apelido_protocolo'] = monitoria['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)

monitoria.loc[:, 'dados_tipo_id'] = monitoria['dados_tipo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'dados_tipo_tipo'] = monitoria['dados_tipo'].apply(lambda x: x['tipo'] if x is not None else None)

monitoria.loc[:, 'dados_status_id'] = monitoria['dados_status'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'ds_descricao'] = monitoria['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)

monitoria = monitoria.drop(['dados_protocolo','dados_status', 'dados_tipo_id','dados_status_id'], axis=1)
monitoria.rename(columns={'dados_protocolo_id': 'id'}, inplace=True)

monitoria.head(2)

dados_tipo data_estimada_filter    id  \
0  {'id': 4, 'tipo': 'Calibração'}           2022-04-28   NaN   
1    {'id': 6, 'tipo': 'Contrato'}           2022-07-06  53.0   

  apelido_protocolo dados_tipo_tipo ds_descricao  
0              None      Calibração    Realizada  
1             Dipro        Contrato    Realizada

In [34]:
monitorias = pd.merge(centros, monitoria, on='id', how='inner')
monitorias

id       apelido_protocolo_x                       numero_protocolo  \
0         4                       BTK                          BTK-COV-202BR   
1         4                       BTK                          BTK-COV-202BR   
2         4                       BTK                          BTK-COV-202BR   
3         4                       BTK                          BTK-COV-202BR   
4         4                       BTK                          BTK-COV-202BR   
...     ...                       ...                                    ...   
16174  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
16175  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
16176  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
16177  2057  Post Market Surveillance  SILIMED – INDÚSTRIA DE IMPLANTES LTDA   
16178  2331            R7508-CVA-2495                                          

          co_externo                                     apelido_centro  \
0      BTK-COV-202BR                                 BTK - Leforte HMCG   
1      BTK-COV-202BR                                 BTK - Leforte HMCG   
2      BTK-COV-202BR                                 BTK - Leforte HMCG   
3      BTK-COV-202BR                                 BTK - Leforte HMCG   
4      BTK-COV-202BR                                 BTK - Leforte HMCG   
...              ...                                                ...   
16174           None     Post Market Surveillance - Centro Vila Olimpia   
16175           None     Post Market Surveillance - Centro Vila Olimpia   
16176           None     Post Market Surveillance - Centro Vila Olimpia   
16177           None     Post Market Surveillance - Centro Vila Olimpia   
16178           None  R7508-CVA-2495 - Maternidade e Cirurgia Nossa ...   

                           dados_tipo data_estimada_filter  \
0         {'id': 1, 'tipo': 'Visita'}           2021-01-05   
1         {'id': 1, 'tipo': 'Visita'}           2021-01-07   
2         {'id': 1, 'tipo': 'Visita'}           2021-01-08   
3         {'id': 1, 'tipo': 'Visita'}           2021-01-12   
4         {'id': 1, 'tipo': 'Visita'}           2021-01-14   
...                               ...                  ...   
16174     {'id': 1, 'tipo': 'Visita'}           2025-09-12   
16175     {'id': 1, 'tipo': 'Visita'}           2025-09-12   
16176     {'id': 1, 'tipo': 'Visita'}           2025-09-12   
16177     {'id': 1, 'tipo': 'Visita'}           2025-09-12   
16178  {'id': 2, 'tipo': 'Monitoria'}           2025-07-04   

            apelido_protocolo_y dados_tipo_tipo ds_descricao  
0                           BTK          Visita    Realizada  
1                           BTK          Visita    Realizada  
2                           BTK          Visita    Realizada  
3                           BTK          Visita    Realizada  
4                           BTK          Visita    Realizada  
...                         ...             ...          ...  
16174  Post Market Surveillance          Visita    Realizada  
16175  Post Market Surveillance          Visita    Realizada  
16176  Post Market Surveillance          Visita    Realizada  
16177  Post Market Surveillance          Visita    Realizada  
16178            R7508-CVA-2495       Monitoria    Realizada  

[16179 rows x 10 columns]

In [35]:
monitorias_filtrado = monitorias[monitorias['dados_tipo_tipo'].str.contains('Monitoria')]
monitorias_filtrado['apelido_protocolo_x'] = monitorias_filtrado['apelido_protocolo_x'].str.strip()

monitorias_filtrado

/tmp/ipykernel_2153/2407812721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monitorias_filtrado['apelido_protocolo_x'] = monitorias_filtrado['apelido_protocolo_x'].str.strip()


id apelido_protocolo_x numero_protocolo   co_externo  \
663      11             DEFLECT        MPZ-II-02    MPZ-II-02   
2374     32               AEGIS      CSL112_3001  CSL112_3001   
2375     32               AEGIS      CSL112_3001  CSL112_3001   
2376     32               AEGIS      CSL112_3001  CSL112_3001   
2721     34             DIA3018     JNJ-28431754          DIA   
...     ...                 ...              ...          ...   
15987  1661         EsSCAPE-996      BIOTEST-996         None   
15988  1661         EsSCAPE-996      BIOTEST-996         None   
16024  1919            SHASTA-5    AROAPOC3-3011         None   
16025  1976         ROXI CAT II   R7508-CAT-2396         None   
16178  2331      R7508-CVA-2495                          None   

                                          apelido_centro  \
663    DEFLECT - Maternidade e Cirurgia Nossa Senhora...   
2374                                AEGIS - Leforte HMCG   
2375                                AEGIS - Leforte HMCG   
2376                                AEGIS - Leforte HMCG   
2721                              DIA3018 - Leforte HMCG   
...                                                  ...   
15987  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...   
15988  EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...   
16024  SHASTA-5 - Maternidade e Cirurgia Nossa Senhor...   
16025  ROXI CAT II - Maternidade e Cirurgia Nossa Sen...   
16178  R7508-CVA-2495 - Maternidade e Cirurgia Nossa ...   

                           dados_tipo data_estimada_filter  \
663    {'id': 2, 'tipo': 'Monitoria'}           2023-05-23   
2374   {'id': 2, 'tipo': 'Monitoria'}           2023-06-26   
2375   {'id': 2, 'tipo': 'Monitoria'}           2023-07-26   
2376   {'id': 2, 'tipo': 'Monitoria'}           2024-01-15   
2721   {'id': 2, 'tipo': 'Monitoria'}           2023-09-05   
...                               ...                  ...   
15987  {'id': 2, 'tipo': 'Monitoria'}           2025-07-24   
15988  {'id': 2, 'tipo': 'Monitoria'}           2025-09-16   
16024  {'id': 2, 'tipo': 'Monitoria'}           2025-04-29   
16025  {'id': 2, 'tipo': 'Monitoria'}           2024-12-18   
16178  {'id': 2, 'tipo': 'Monitoria'}           2025-07-04   

      apelido_protocolo_y dados_tipo_tipo ds_descricao  
663               DEFLECT       Monitoria    Realizada  
2374                AEGIS       Monitoria    Realizada  
2375                AEGIS       Monitoria    Realizada  
2376                AEGIS       Monitoria    Realizada  
2721              DIA3018       Monitoria    Realizada  
...                   ...             ...          ...  
15987         EsSCAPE-996       Monitoria    Realizada  
15988         EsSCAPE-996       Monitoria    Realizada  
16024            SHASTA-5       Monitoria    Realizada  
16025         ROXI CAT II       Monitoria    Realizada  
16178      R7508-CVA-2495       Monitoria    Realizada  

[290 rows x 10 columns]

In [36]:
def verificar_monitorias_pendentes():
    monitorias_filtrado.loc[:, 'data_estimada_filter'] = pd.to_datetime(monitorias_filtrado['data_estimada_filter'])
    hoje = datetime.today()

    monitorias_pendentes = monitorias_filtrado[
        (monitorias_filtrado['ds_descricao'] == 'Pendente') &
        (monitorias_filtrado['data_estimada_filter'] >= hoje)
    ]
    monitorias_pendentes = monitorias_pendentes.sort_values(by='data_estimada_filter', ascending=True)

    print(monitorias_pendentes[['dados_tipo_tipo', 'data_estimada_filter', 'apelido_protocolo_x', 'ds_descricao','apelido_centro']])
    return monitorias_pendentes

schedule.every().monday.at("09:00").do(verificar_monitorias_pendentes)

monitorias_pendentes_df = verificar_monitorias_pendentes()

      dados_tipo_tipo data_estimada_filter  apelido_protocolo_x ds_descricao  \
12712       Monitoria  2025-09-22 00:00:00  EASi-HF - 1378-0020     Pendente   
13457       Monitoria  2025-09-23 00:00:00             EXHALE-2     Pendente   
12473       Monitoria  2025-09-26 00:00:00         TAK-330-3001     Pendente   
15751       Monitoria  2025-09-29 00:00:00                  DLG     Pendente   
11715       Monitoria  2025-09-30 00:00:00             EXHALE-2     Pendente   
13504       Monitoria  2025-10-06 00:00:00           MK0616-015     Pendente   
10376       Monitoria  2025-10-08 00:00:00  VICTORION-1-PREVENT     Pendente   
7157        Monitoria  2025-10-10 00:00:00      LIBREXIA-STROKE     Pendente   
15721       Monitoria  2025-10-15 00:00:00   LATAM LOWERS LDL-C     Pendente   
12472       Monitoria  2025-10-23 00:00:00         TAK-330-3001     Pendente   
13502       Monitoria  2025-10-28 00:00:00           MK0616-015     Pendente   
12134       Monitoria  2025-10-30 00:00:

In [37]:
envio_para = os.getenv('ENVIO_PARA')

In [38]:
monitorias_pendentes_df.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_tipo_tipo': 'Monitoria', 'data_estimada_filter': 'Data Estimada',
                                        'ds_descricao': 'Status'}, inplace=True)
monitorias_pendentes_df

id  apelido_protocolo_x numero_protocolo co_externo  \
12712   655  EASi-HF - 1378-0020        1378-0020       None   
13457  1181             EXHALE-2     AR-DEX-22-01       None   
12473   468         TAK-330-3001     TAK-330-3001       None   
15751  1531                  DLG                        None   
11715   371             EXHALE-2     AR-DEX-22-01       None   
13504  1182           MK0616-015      MK0616-015        None   
10376   300  VICTORION-1-PREVENT    CKJX839D12302       None   
7157    169      LIBREXIA-STROKE  70033093STR3001     STRUCK   
15721  1495   LATAM LOWERS LDL-C    CKJX839A1MX02       None   
12472   468         TAK-330-3001     TAK-330-3001       None   
13502  1182           MK0616-015      MK0616-015        None   
12134   397              UCESIVE      PB016-03-01       None   
12517   491               GLORIA      OBI-822-011      PT491   
15720  1495   LATAM LOWERS LDL-C    CKJX839A1MX02       None   
13503  1182           MK0616-015      MK0616-015        None   
12471   468         TAK-330-3001     TAK-330-3001       None   
12133   397              UCESIVE      PB016-03-01       None   
7158    169      LIBREXIA-STROKE  70033093STR3001     STRUCK   

                                           Estudo/Centro  \
12712  EASi-HF - 1378-0020 - Maternidade e Cirurgia N...   
13457                    EXHALE-2 - Santa Casa de Santos   
12473                TAK-330-3001 - Santa Casa de Santos   
15751                         DLG - Santa Casa de Santos   
11715  EXHALE-2 - Maternidade e Cirurgia Nossa Senhor...   
13504  MK0616-015  - Maternidade e Cirurgia Nossa Sen...   
10376                 VICTORION-1-PREVENT - Leforte HMCG   
7157   LIBREXIA-STROKE - Maternidade e Cirurgia Nossa...   
15721  LATAM LOWERS LDL-C - Maternidade e Cirurgia No...   
12472                TAK-330-3001 - Santa Casa de Santos   
13502  MK0616-015  - Maternidade e Cirurgia Nossa Sen...   
12134                     UCESIVE - Santa Casa de Santos   
12517  GLORIA - Maternidade e Cirurgia Nossa Senhora ...   
15720  LATAM LOWERS LDL-C - Maternidade e Cirurgia No...   
13503  MK0616-015  - Maternidade e Cirurgia Nossa Sen...   
12471                TAK-330-3001 - Santa Casa de Santos   
12133                     UCESIVE - Santa Casa de Santos   
7158   LIBREXIA-STROKE - Maternidade e Cirurgia Nossa...   

                           dados_tipo        Data Estimada  \
12712  {'id': 2, 'tipo': 'Monitoria'}  2025-09-22 00:00:00   
13457  {'id': 2, 'tipo': 'Monitoria'}  2025-09-23 00:00:00   
12473  {'id': 2, 'tipo': 'Monitoria'}  2025-09-26 00:00:00   
15751  {'id': 2, 'tipo': 'Monitoria'}  2025-09-29 00:00:00   
11715  {'id': 2, 'tipo': 'Monitoria'}  2025-09-30 00:00:00   
13504  {'id': 2, 'tipo': 'Monitoria'}  2025-10-06 00:00:00   
10376  {'id': 2, 'tipo': 'Monitoria'}  2025-10-08 00:00:00   
7157   {'id': 2, 'tipo': 'Monitoria'}  2025-10-10 00:00:00   
15721  {'id': 2, 'tipo': 'Monitoria'}  2025-10-15 00:00:00   
12472  {'id': 2, 'tipo': 'Monitoria'}  2025-10-23 00:00:00   
13502  {'id': 2, 'tipo': 'Monitoria'}  2025-10-28 00:00:00   
12134  {'id': 2, 'tipo': 'Monitoria'}  2025-10-30 00:00:00   
12517  {'id': 2, 'tipo': 'Monitoria'}  2025-11-03 00:00:00   
15720  {'id': 2, 'tipo': 'Monitoria'}  2025-11-12 00:00:00   
13503  {'id': 2, 'tipo': 'Monitoria'}  2025-11-17 00:00:00   
12471  {'id': 2, 'tipo': 'Monitoria'}  2025-11-27 00:00:00   
12133  {'id': 2, 'tipo': 'Monitoria'}  2025-12-02 00:00:00   
7158   {'id': 2, 'tipo': 'Monitoria'}  2025-12-08 00:00:00   

       apelido_protocolo_y  Monitoria    Status  
12712  EASi-HF - 1378-0020  Monitoria  Pendente  
13457             EXHALE-2  Monitoria  Pendente  
12473         TAK-330-3001  Monitoria  Pendente  
15751                  DLG  Monitoria  Pendente  
11715             EXHALE-2  Monitoria  Pendente  
13504          MK0616-015   Monitoria  Pendente  
10376  VICTORION-1-PREVENT  Monitoria  Pendente  
7157       LIBREXIA-STROKE  Monitoria  Pendente  
15721   LATAM LOWERS L

In [39]:
def enviar_emails(monitorias_pendentes_df):
    global envio_para
    try:
        if monitorias_pendentes_df is None or monitorias_pendentes_df.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        tabela_html = monitorias_pendentes_df[['Estudo/Centro', 'Monitoria', 'Data Estimada', 'Status']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Monitoria Pendentes"

        # Validação da lista de destinatários
        if isinstance(envio_para, list):
            # Expressão regex para validar e-mails
            email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
            envio_para = [email.strip() for email in envio_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            envio_para = []

        # Verifica se há destinatários antes de enviar
        destinatarios = [email_usuario] + envio_para
        if not destinatarios:
            print("Nenhum destinatário válido encontrado. O e-mail não foi enviado.")
            return

        # Somente adiciona Bcc se houver destinatários válidos
        if envio_para:
            msg['Bcc'] = ', '.join(envio_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de monitoria programadas para os pr&oacute;ximos dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            # Envio corrigido para passar apenas destinatários válidos
            server.sendmail(email_usuario, destinatarios, msg.as_string())

        print("E-mail de visitas pendentes enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(monitorias_pendentes_df)

E-mail de visitas pendentes enviado com sucesso!
